In [236]:
# pip install --upgrade imutils


In [4]:
import pandas as pd
import cv2
import glob
import matplotlib.pyplot as plt
import imutils
from imutils.contours import sort_contours
import numpy as np
from PIL import Image
import os



In [5]:
TrainSet = pd.read_csv("TrainLables.csv").drop(['Unnamed: 0'],axis=1)
TestSet = pd.read_csv("TestLabels.csv").drop(['Unnamed: 0'],axis=1)

TrainSet.head()

,Image,Text
0,AHTD3A0001_Para2_3.png,من العذاب في الآخرة وأفضل ما رزقهم الله تعالى ...
1,AHTD3A0001_Para2_4.png,لجميع الأشياء والذي لا يقدر أحد في الدنيا على ...
2,AHTD3A0001_Para3_1.png,فقال له إنك في منازل آبائك وأجدادك من الجبابر...
3,AHTD3A0001_Para3_2.png,قبلك وشيدوه دونك وبنوا القلاع والحصون ومهدوا ا...
4,AHTD3A0002_Para2_1.png,وكذلك طالب الآخرة مجتهد في العمل المنجي به روح...


In [6]:
Train_Images = [cv2.imread('Line Images/'+Name) for Name in TrainSet['Image']]
Test_Images = [cv2.imread('Line Images/'+Name) for Name in TestSet['Image']]


In [7]:
Train_Labels = TrainSet['Text']
Test_Labels = TestSet['Text']


In [8]:
def augment_image(img,label):
    # Making directory for saving the augmented images along with the original image
    
    dirname = 'AugmentedImages'
    os.makedirs('AugmentedImages', exist_ok=True)  
    #Saving the original Image
    path0 = 'AugmentedImages/' + label
    cv2.imwrite(path0,img)
    
    #saving the blurred image
    blurred = cv2.GaussianBlur(img,(91,91),cv2.BORDER_DEFAULT)
    path1 = 'AugmentedImages/blurred' +label
    cv2.imwrite(path1,blurred)
    
    #saving the edged image
    edged = cv2.Canny(blurred, 30, 150)
    path2 = 'AugmentedImages/edged' +label
    cv2.imwrite(path2,edged)
    
    #saving the bleed through enhanced img
    
    dst = cv2.fastNlMeansDenoisingColored(img,None,100,100,7,21)
    path3 = 'AugmentedImages/bte' +label
    cv2.imwrite(path3,dst)
    
    #saving the 25% resolution off
    
    PIL_image = Image.fromarray(np.uint8(img)).convert('RGB')
    
    path4 = 'AugmentedImages/lq' +label
    PIL_image.save(path4, quality=75)
    
    #saving the contoured version of the text line
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  

    #--- performing Otsu threshold ---
    ret,thresh1 = cv2.threshold(gray, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)

    #--- choosing the right kernel
    #--- kernel size of 3 rows (to join dots above letters 'i' and 'j')
    #--- and 10 columns to join neighboring letters in words and neighboring words
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (70, 20))
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    im2 = img.copy()
    for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w>100 and h>30:
                cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 3)    
    
    path5 = 'AugmentedImages/contoured' +label
    cv2.imwrite(path5,im2)

In [9]:
for i in range(len(Train_Images)):
    augment_image(Train_Images[i],TrainSet['Image'][i])